In [1]:
# This is Main function.
# Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob
# Streaming With Tweepy 
# http://docs.tweepy.org/en/v3.4.0/streaming_how_to.html#streaming-with-tweepy


# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) \
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False


In [2]:
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [3]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="password",
    database="TwitterDB",
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()
    



In [ ]:
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOEKN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(track = settings.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()

RT @dillonliam: In Manhattan Beach, a woman painted her Airbnb rental with emojis because its time for the emoji. 

Her neighbors say th
Long: None, Lati: None
RT @dillonliam: In Manhattan Beach, a woman painted her Airbnb rental with emojis because its time for the emoji. 

Her neighbors say th
Long: None, Lati: None
Unless those Airbnb guests posed a threat to the neighborhood, the neighbors should have left it alone.  Theres al https://t.co/sKfxTNZQXy
Long: None, Lati: None
...

It's rare case!
airbnb 

Long: None, Lati: None
@TwistedKelpie @snpwatch Are these intended to be AirBnB lets and removed from the local housing stock so (young, l https://t.co/eKR6DEFpFN
Long: None, Lati: None
Like Airbnb but for Health and Human Services
Long: None, Lati: None
RT @fabiomoioli: U.S. technology companies founded by 1st and 2nd generation immigrants: 

Apple
Google
Amazon
Facebook
Oracle
IBM
Uber
Air
Long: None, Lati: None
RT @dillonliam: In Manhattan Beach, a woman painted her Airbnb rental 

RT @CiaranPolicy: Top work from @Effy_Yeomans &amp; @thetimesscot.
"In one residential development in Western Harbour, Leith, a lettings compan
Long: None, Lati: None
RT @xmo999onx: Willa Mnich - Studio Fiolek 
 -  .

      .
      
Long: None, Lati: None
Hi wei irgendwer wie man billig in Mnchen bernachten kann ich bin dieses Jahr 3 mal da und brauche echt ne billi https://t.co/DjLbv9Y1e9
Long: None, Lati: None
Has anyone done #Airbnb in #Tokyo? Any recommendations on areas?
Long: None, Lati: None
RT @Loopsidernews: Vous connaissez AirBnb, mais connaissez-vous le FairBnb ? 
Cette cooprative cherche notamment  rduire les impacts ng
Long: None, Lati: None
RT @ZenConciergerie: Un airbnb Harry Potter a ouvert ses portes en France - https://t.co/S9A1REOzmJ https://t.co/Pwy1x5A3Z6 https://t.co/EH
Long: None, Lati: None
RT @makiwi: Proof that @Airbnb is an American company: there is no search criteria for close to public transportation. Glaring oversight,
Long: None, Lati: None
RT @dillonlia

Tell my why I wrote my momma a glowing review on AIRBNB and this lady cant even answer my calls. 
Long: None, Lati: None
RT @dillonliam: In Manhattan Beach, a woman painted her Airbnb rental with emojis because its time for the emoji. 

Her neighbors say th
Long: None, Lati: None
RT @dillonliam: In Manhattan Beach, a woman painted her Airbnb rental with emojis because its time for the emoji. 

Her neighbors say th
Long: None, Lati: None
RT @Effy_Yeomans: One tenant I spoke to was served an eviction notice forcing him to move out the day before New Year's eve. His old home n
Long: None, Lati: None
Great marketing for the beach! Millennials would love to have this Airbnb American beach towns always seem to be s https://t.co/PqI6hRxAJc
Long: None, Lati: None
Hopefully we all agree on a house tomorrow for the Airbnb in Rosarito because that way I can purchase my ticket next week already.
Long: None, Lati: None
imagine being this pressed over what your neighbors do, rich people are nuts
Lon

@GhostTeas There's an airbnb near matsuri idk if that's cheaper bc ive never booked a hotel or airbnb but its right https://t.co/R95ETVLpHj
Long: None, Lati: None
Somebody lets rent an AIRBNB &amp; play Bad Girls Club
Long: None, Lati: None
@alexalucina Everyone who is taking the landladys side doesnt know the whole story. Apparently the owner of the e https://t.co/7eFMcJsLpU
Long: None, Lati: None
RT @BoraOzkent: 60 Saniyede  Dijital Dnyada Olanlar (2019) 

188,000,000 Email
4,500,000 Yotube zleme
4,497,420 Google Aramas
694,444
Long: None, Lati: None
RT @DenverStrippers: We bring the strip club  to you ! 
 Nude Female Stripperse, topless bartenders
 Adult Entertainment at your AIRBNB any
Long: None, Lati: None
https://t.co/P6FvVPIPx6
Long: None, Lati: None
RT @ZenConciergerie: Un airbnb Harry Potter a ouvert ses portes en France - https://t.co/S9A1REOzmJ https://t.co/Pwy1x5A3Z6 https://t.co/EH
Long: None, Lati: None
RT @GoddessDes2: Im getting an airbnb for my vacation in Orlando, i 

A homeowner reported her neighbors AirBnB. The neighbor retaliated with emojis. - Los Angeles Times https://t.co/xOXIGmiy3j
Long: None, Lati: None
RT @indiobarcelona: Qu queremos?
Ms bicis!
Cmo las queremos?
Desreguladas!
Y para quines las queremos?
... https://t.co/jbUu1xiUAw
Long: None, Lati: None
RT @fabiomoioli: U.S. technology companies founded by 1st and 2nd generation immigrants: 

Apple
Google
Amazon
Facebook
Oracle
IBM
Uber
Air
Long: None, Lati: None
RT @Rhythmtrader: If you want the truth:

Don't ask a banker 
about bitcoin.

Don't ask a taxi driver  
about Uber.

Don't ask a mailcarr
Long: None, Lati: None
https://t.co/15NeRYSeIz
Long: None, Lati: None
@TreyAllensworth @westelm @Equinox @virginhotels @MontageHotels @Amanresorts @Expedia @Airbnb Looky here! 
https://t.co/qfTFdMIosY
Long: None, Lati: None
This Striking Joshua Tree Airbnb NEEDS to be on Your Travel Bucket List, Period https://t.co/oi2PZgw82L
Long: None, Lati: None
Seriously, these neighbors need to grow up. Asi

...

New to airbnb? Register using my link and Get $40 Off Your Next Airbnb &gt;&gt; https://t.co/Oyo5fVo1Wm via visitsamerica https://t.co/tMpmGjGNtz
Long: None, Lati: None
@neoncutoff in what context? AirBnb type sitch? or HotPads aka I'm leasing for a year+
Long: None, Lati: None
Fancy a week in Paris?
Beautiful flat in Paris 11th District for rent during August 2019, sleeps 4 people. Minimum https://t.co/UmQt6lgo4h
Long: None, Lati: None
RT @420Cyber: Airbnb introduces new search tools for business travelers https://t.co/1mBZuMDwph https://t.co/1g0Wvxge8P
Long: None, Lati: None
$4,497 a person https://t.co/SzziOrIg0z
Long: None, Lati: None
123...airbnbhttps://t.co/12gYd4tnyC
Long: None, Lati: None
Veja esta casa na Airbnb! https://t.co/XGgm48MPyA
Long: None, Lati: None
I love how well this captures the bizarre entitlement of neighbors thinking they can control homes they don't own
Long: None, Lati: None
Airbnb Coupon Code 
Save $40 off first booking 
Referral link  https://t.co/dfxHW0M6

Who loves hosting a dinner party but hates cooking and cleaning? Me! Check out @JodieMlikota's startup @Gatharing - https://t.co/rwhG2ZbaSq
Long: None, Lati: None
RT @firyalblfst: Jadi abis iseng searching di airbnb dan nemu penginapan2 lucu dan cozy bgt di daerah Bandung! 
Cusss ini dia penginapan su
Long: None, Lati: None
Est noche dormir en un Airbnb donde hay 2 perritos. Excelente servicio.
Long: None, Lati: None
RT @AlbertoVill: Est noche dormir en un Airbnb donde hay 2 perritos. Excelente servicio.
Long: None, Lati: None
RT @AlbertoVill: Est noche dormir en un Airbnb donde hay 2 perritos. Excelente servicio.
Long: None, Lati: None
Are we seriously such a pathetic society that we complain to the government about paint jobs?
Long: None, Lati: None
RT @AlbertoVill: Est noche dormir en un Airbnb donde hay 2 perritos. Excelente servicio.
Long: None, Lati: None
RT @dillonliam: In Manhattan Beach, a woman painted her Airbnb rental with emojis because its time for the emoji. 

Her neighb

im literally looking at hotels/airbnbs rn fully knowing that i have 5 in my bank acc rn
Long: None, Lati: None
Airbnb introduces new search tools for business travelers https://t.co/iOlMjdwsth via @circleboom
Long: None, Lati: None
RT @Effy_Yeomans: Tenants are evicted to make way for holiday homes, as one in ten city centre properties is listed on Airbnb: my investiga
Long: None, Lati: None
RT @q1fxys: Kalau korang ada duit lebih, cubalah stay dekat AirBnB dia. Semua ni ada private pool okey. Dahla murah. Iskkkk https://t.co/f0
Long: None, Lati: None
RT @6blogger: Fuck! German berbabe @AnnyAuroraPorn pays for Airbnb with SEX!  

(Sehr geil Anny, du darfst jederzeit bei mir auch bernach
Long: None, Lati: None
CUPOM DE DESCONTO

IFOOD:
W75ZJ7CYEM  - 12 reais no seu primeiro pedido

RAPPI:
5dl3924467 -  30 reais  + 150 em fr https://t.co/dyxNYOKnpb
Long: None, Lati: None
RT @JimmyDeuxFois__: En vrai, la seule soire entre Twittos  laquelle jai particip tait un chec mdr. Elles avaient lou 

RT @q1fxys: Kalau korang ada duit lebih, cubalah stay dekat AirBnB dia. Semua ni ada private pool okey. Dahla murah. Iskkkk https://t.co/f0
Long: None, Lati: None
Check out this home on Airbnb! https://t.co/fjoytssUOC
Long: None, Lati: None
RT @TaliShapiro: 8| @archenemymetal @FIFAWorldCup @UEFA @puma @Airbnb @bookingcom @TripAdvisor @Expedia and Red Sea Jazz Festival participa
Long: None, Lati: None
A homeowner reported her neighbors AirBnB. The neighbor retaliated with emojis. - Los Angeles Times this is exactl https://t.co/cGtK9aG1Bk
Long: None, Lati: None
@GenkaiKuroneko @itsmtoyou @Tiara_is_TMI @KissChattanooga @JiminsBookmark @Sweetdestruxion @haeandmoon https://t.co/RhZ6mnrlFP
Long: None, Lati: None
RT @mylegalhotel: Airbnb is not the nice home sharing platform it wants us to believe, it is a nasty cheating stealing deceptive platform
Long: None, Lati: None
RT @JackBoyMuc: If u need anything paid online (rent, bills, court tickets, flights, rentals, phone bill, car note, car ins

RT @dillonliam: In Manhattan Beach, a woman painted her Airbnb rental with emojis because its time for the emoji. 

Her neighbors say th
Long: None, Lati: None
RT @pablofelices: Como dice el @teamloserok ample el. kiosquito, segu este link y proba Airbnb, Regstrate a travs de mi enlace y conseg
Long: None, Lati: None
I feel the same way about people with pro-Trump signs and bumper stickers.
Long: None, Lati: None
Check out this home on Airbnb! https://t.co/HHQtmOMQFG
Long: None, Lati: None
so,,, would any moots wanna maybe,,, travel to cali with me? im seriously considering going, and am airbnb is chea https://t.co/oMNtYY9YDy
Long: None, Lati: None
RT @dillonliam: In Manhattan Beach, a woman painted her Airbnb rental with emojis because its time for the emoji. 

Her neighbors say th
Long: None, Lati: None
Sina News wurde gerade verffentlicht! https://t.co/PI3aaRWLul #linnemann #airbnb
Long: None, Lati: None
Die Zerstrung des Wohnraums fr Einheimische durch AirBnB ist nichts Neues, schr